In [34]:
"""Define a custom Reasoning and Action agent.

Works with a chat model with tool calling support.
"""

import asyncio
import json
import sys
import os, re
from dotenv import load_dotenv
import subprocess

load_dotenv()
from langgraph.checkpoint.memory import MemorySaver

# sys.path.append("../")
sys.path.append(r"/home/zt/workspace/MooseAgent/src")
from tqdm import tqdm
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import ToolNode
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import Chroma, FAISS
from mooseagent.configuration import Configuration
from langchain_openai import OpenAIEmbeddings
from mooseagent.utils import (
    BGE_M3_EmbeddingFunction,
    tran_dicts_to_str,
    extract_files_and_descriptions,
    extract_sub_tasks,
)
from mooseagent.state1 import FlowState, ReviewState, OneFileState, ExtracterFileState, ArchitectState
from mooseagent.utils import load_chat_model
from mooseagent.prompts1 import (
    SYSTEM_ALIGNMENT_PROMPT,
    HUMAN_ALIGNMENT_PROMPT,
    SYSTEM_REVIEW_WRITER_PROMPT,
    SYSTEM_ARCHITECT_PROMPT,
    # HUMAN_ARCHITECT_PROMPT,
)
from mooseagent.write_module import bulid_writer_module
from langgraph.constants import Send
from langgraph.types import interrupt, Command
from langchain.tools.retriever import create_retriever_tool

config = RunnableConfig()
configuration = Configuration.from_runnable_config(config)
embedding_function = OpenAIEmbeddings() if configuration.embedding_function == "OPENAI" else BGE_M3_EmbeddingFunction()
batch_size = configuration.batch_size
top_k = configuration.top_k
json_file = configuration.rag_json_path
try:
    vectordb_input = FAISS.load_local(
        configuration.input_database_path, embedding_function, allow_dangerous_deserialization=True
    )
    retriever_input = vectordb_input.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
    vectordb_dp = FAISS.load_local(
        configuration.dp_database_path, embedding_function, allow_dangerous_deserialization=True
    )
    retriever_dp = vectordb_dp.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
    # define tools
    tools = [
        create_retriever_tool(
            retriever_input,
            "retrieve_moose_case",
            "Search and return information about MOOSE simulation cases that are relevant to the simulation case you are working on.",
        ),
        create_retriever_tool(
            retriever_dp,
            "retrieve_moose_dp",
            "Search and return information about the document of MOOSE app that are relevant to the simulation case you are working on.",
        ),
    ]
except Exception as e:
    print(f"Error loading vector database: {e}")

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

In [ ]:
configuration = Configuration.from_runnable_config(config)


def helper(state: State):
    llm = load_chat_model(configuration.review_writer_model).bind_tools(tools)
    return {"messages": [llm.invoke(state["messages"])]}


# helper.invoke("How to define Dirichlet boundary condition in MOOSE?")

In [37]:
graph_builder = StateGraph(State)
graph_builder.add_node("helper", helper)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge("tools", "helper")
graph_builder.set_entry_point("helper")
graph_builder.add_conditional_edges(
    "helper",
    tools_condition,
)
graph = graph_builder.compile()

In [38]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [39]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [ ]:
answer = graph.invoke(
    {"messages": [{"role": "user", "content": "How to define Dirichlet boundary condition in MOOSE?"}]}
)

In [ ]:
print(answer["messages"][3].content)

IndexError: list index out of range

In [40]:
stream_graph_updates("How to define Dirichlet boundary condition in MOOSE?")

Assistant: To define a Dirichlet boundary condition in MOOSE, you typically use the `DirichletBC` class. This class is used to enforce a fixed value (usually zero) on a specified boundary for a given variable. Here’s a step-by-step guide on how to define a Dirichlet boundary condition in a MOOSE input file:

### 1. **Define the Variable:**
   First, you need to define the variable on which you want to apply the Dirichlet boundary condition. This is usually done in the `Variables` block of the input file.

   ```yaml
   [Variables]
     [u]
       order = FIRST
       family = LAGRANGE
     []
   []
   ```

### 2. **Define the Boundary Condition:**
   Next, you define the Dirichlet boundary condition in the `Kernels` block. You specify the variable, the boundary where the condition should be applied, and the value to be enforced.

   ```yaml
   [BCs]
     [left]
       type = DirichletBC
       variable = u
       boundary = left
       value = 0
     []
   []
   ```

   - `type`: Speci